<a href="https://colab.research.google.com/github/trantoai1/GraduatedProject/blob/main/Evaluate_pruned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/SSL/monodepth2
from __future__ import absolute_import, division, print_function
%matplotlib inline

import os
import numpy as np
import PIL.Image as pil
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from torchvision import transforms
import time
import networks
from utils import download_model_if_doesnt_exist

/content/drive/MyDrive/SSL/monodepth2


## Setting up network and loading weights

In [ ]:
pruned_name = "models"
prunedpath = "/content/drive/MyDrive/SSL/monodepth2/testprune/mono_640x192"

pruned_path = os.path.join(prunedpath, pruned_name, "weights_1/encoder.pth")
# LOADING PRETRAINED MODEL
pruned_model = networks.ResnetEncoder(18, False)
loaded_dict_enc = torch.load(pruned_path, map_location='cpu')
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in pruned_model.state_dict()}
pruned_model.load_state_dict(filtered_dict_enc)
pruned_model.eval()

In [ ]:
model_name = "mono_640x192"
modelpath = "/content/drive/MyDrive/SSL/monodepth2/models"
encoder_path = os.path.join(modelpath, model_name, "encoder.pth")
# LOADING PRETRAINED MODEL
encoder = networks.ResnetEncoder(18, False)
loaded_dict_enc = torch.load(encoder_path, map_location='cpu')
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
encoder.load_state_dict(filtered_dict_enc)
encoder.eval()

## Loading the test image and preprocessing

In [32]:
total = 0
zeros = 0
for m in encoder.modules():
    if isinstance(m, nn.Conv2d):
        total += m.weight.data.numel()
        weight_copy = m.weight.data.abs().clone()
        mask = weight_copy.gt(0).float()
        zeros = zeros + mask.numel() - torch.sum(mask)
print('total number weight:{}'.format(total))
print('total zeros weight:{}'.format(zeros))
print('total remain weight:{}'.format(total-zeros))



total number weight:11166912
total zeros weight:0.0
total remain weight:11166912.0


In [35]:
total = 0
pruned = 0
for m in pruned_model.modules():
    if isinstance(m, nn.Conv2d):
        total += m.weight.data.numel()
        weight_copy = m.weight.data.abs().clone()
        mask = weight_copy.gt(0).float()
        pruned = pruned + mask.numel() - torch.sum(mask)
print('total number weight:{}'.format(total))
print('total zeros weight:{}'.format(pruned))
print('total remain weight:{}'.format(total-pruned))
print('Pruned rate:%2f'% (100*int(pruned.item())/total))

total number weight:11166912
total zeros weight:10776676.0
total remain weight:390236.0
Pruned rate:96.505426
